In [1]:
import os
import datetime
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# sklearn
from sklearn.model_selection import train_test_split
from utils import get_merged_df

In [2]:
# tensorflow
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

import keras
from keras.models import Sequential, Model  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import F1Score, AUC, CategoricalAccuracy, BinaryAccuracy
from tensorflow.keras.optimizers import RMSprop

print( f'tf.__version__: {tf.__version__}' )
print( f'keras.__version__: {keras.__version__}' )


tf.__version__: 2.15.0
keras.__version__: 2.15.0


In [3]:
import cv2
from PIL import Image

In [4]:
data_dir = 'training_data/training_data'
norm_csv_path = 'training_data/training_norm.csv'
cleaned_df = get_merged_df(data_dir, norm_csv_path)

len(cleaned_df)

13792

In [5]:
# angle_labels = cleaned_df['angle'].to_list()
# speed_labels = cleaned_df['speed'].to_list()
# image_paths = cleaned_df['image_path'].to_list()

X_train, X_valid, y_train, y_valid = train_test_split(cleaned_df['image_path'].to_list(), cleaned_df['angle'].to_list(), test_size=0.3)

# X_train, X_valid, angle_train, angle_valid, speed_train, speed_valid = train_test_split(image_paths, angle_labels, speed_labels, test_size=0.3)
print("Training data: %d\nValidation data: %d" % (len(X_train), len(X_valid)))

# print(type(angle_labels))

Training data: 9654
Validation data: 4138


In [6]:
def my_imread(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def img_preprocess(image):
    # height, _, _ = image.shape
    # image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relavant for lane following
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    # image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (224,224)) # input image size (200,66) Nvidia model
    # image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
    # image = (image - 127.5) / 127.5
    return image


# fig, axes = plt.subplots(1, 2, figsize=(15, 10))
# image_orig = my_imread(merged_df['image_path'][image_index])
# image_processed = img_preprocess(image_orig)
# axes[0].imshow(image_orig)
# axes[0].set_title("orig")
# axes[1].imshow(image_processed)
# axes[1].set_title("processed")

In [7]:
def image_data_generator(image_paths, angle_labels, batch_size):
    while True:
        batch_images = []
        batch_angles = []

        for i in range(batch_size):
            random_index = random.randint(0, len(image_paths) - 1)
            image = my_imread(image_paths[random_index])
            angle_label = angle_labels[random_index]
            angle_label *= 16

            image = img_preprocess(image)
            batch_images.append(image)

            angle_one_hot = to_categorical(angle_label, num_classes=17)
            batch_angles.append(angle_one_hot)

        yield( np.asarray(batch_images), np.asarray(batch_angles))

In [17]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary(show_trainable=True)

83683744/83683744 [==============================] - 3s 0us/step
Model: "model_5"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_7 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling_5 (Rescaling)     (None, 150, 150, 3)       0         Y          
                                                                            
 xception (Functional)       (None, 5, 5, 2048)        2086148   N          
                                                       0                    
                                                                            
 global_average_pooling2d_5  (None, 2048)              0         Y          
  (GlobalAveragePooling2D)                                                  
                                                                       

In [23]:
def mobile_net_classification_model():
    base_model = keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_shape=(224,224,3))
    base_model.trainable = False
    
    inputs = Input(shape=(224, 224, 3))
    x = tf.keras.layers.Rescaling(1./127.5, offset=-1)(inputs)
    x = base_model(x, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)

    # Common part of the model
    common = Dense(1024, activation='relu')(x)
    common = Dropout(0.3)(common)

    # Branch for the angle prediction (multi-class classification)
    angle_branch = Dense(512, activation='relu')(common)
    angle_branch = Dropout(0.3)(angle_branch)
    angle_output = Dense(17, activation='softmax', name='angle_output')(angle_branch) # 17 classes for angle

    model = Model(inputs=inputs, outputs=angle_output)
    # Create an RMSprop optimizer with a custom learning rate
    custom_lr = 0.001  # Example custom learning rate
    optimizer = RMSprop(learning_rate=custom_lr)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics='accuracy')

    return model

# model = nvidia_model()
model = mobile_net_classification_model()
model.summary(show_trainable=True)

Model: "model_9"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         Y          
                                                                            
 rescaling_9 (Rescaling)     (None, 224, 224, 3)       0         Y          
                                                                            
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   N          
 tional)                                                                    
                                                                            
 global_average_pooling2d_9  (None, 1280)              0         Y          
  (GlobalAveragePooling2D)                                                  
                                                                            
 dense_17 (Dense)            (None, 1024)              1311

In [9]:
model_output_dir = 'models/angle'

# start Tensorboard before model fit, so we can see the epoch tick in Tensorboard
# Jupyter Notebook embedded Tensorboard is a new feature in TF 2.0!!  

# clean up log folder for tensorboard
log_dir_root = f'{model_output_dir}/logs'
#!rm -rf $log_dir_root

tensorboard_callback = TensorBoard(log_dir_root, histogram_freq=1)

# Specify the file path where you want to save the model
filepath = 'models/angle/{epoch:02d}-{val_loss:.2f}'

# Create the ModelCheckpoint callback
model_checkpoint_callback = ModelCheckpoint(
    filepath,
    monitor='val_loss',     # Monitor validation loss
    verbose=1,              # Log a message each time the callback saves the model
    save_best_only=True,    # Only save the model if 'val_loss' has improved
    save_weights_only=False, # Only save the weights of the model
    mode='min',             # 'min' means the monitored quantity should decrease
    save_freq='epoch'       # Check every epoch
)

In [14]:
history = model.fit(
    image_data_generator(X_train, y_train, batch_size=128),
    steps_per_epoch=len(X_train) // 128,
    epochs=10,
    validation_data = image_data_generator(X_valid, y_valid, batch_size=128),
    validation_steps=len(X_train) // 128,
    verbose=1,
    shuffle=1,
    callbacks=[model_checkpoint_callback, tensorboard_callback]
)

Epoch 1/10
75/75 [==============================] - ETA: 0s - loss: 2.2537 - accuracy: 0.2717
Epoch 1: val_loss improved from inf to 1.80615, saving model to models/angle/01-1.81
INFO:tensorflow:Assets written to: models/angle/01-1.81/assets


INFO:tensorflow:Assets written to: models/angle/01-1.81/assets


75/75 [==============================] - 169s 2s/step - loss: 2.2537 - accuracy: 0.2717 - val_loss: 1.8062 - val_accuracy: 0.2915
Epoch 2/10
11/75 [===>..........................] - ETA: 1:14 - loss: 1.7200 - accuracy: 0.3125

KeyboardInterrupt: 

In [18]:
model.trainable = True
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['rescaling_1[0][0]']         
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                        

In [19]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)  # Lower learning rate
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 3: Continue training the model
history_fine = model.fit(
    image_data_generator(X_train, y_train, batch_size=128),
    steps_per_epoch=len(X_train) // 128,
    epochs=10,  # You can adjust the number of epochs for fine-tuning
    validation_data=image_data_generator(X_valid, y_valid, batch_size=128),
    validation_steps=len(X_valid) // 128,
    verbose=1,
    callbacks=[model_checkpoint_callback]  # Assuming this callback is already defined
)

Epoch 1/10
75/75 [==============================] - ETA: 0s - loss: 1.4199 - accuracy: 0.4320
Epoch 1: val_loss did not improve from 1.42729
75/75 [==============================] - 740s 10s/step - loss: 1.4199 - accuracy: 0.4320 - val_loss: 1.7983 - val_accuracy: 0.4062
Epoch 2/10
75/75 [==============================] - ETA: 0s - loss: 1.1333 - accuracy: 0.5450 
Epoch 2: val_loss did not improve from 1.42729
75/75 [==============================] - 806s 11s/step - loss: 1.1333 - accuracy: 0.5450 - val_loss: 1.7578 - val_accuracy: 0.3875
Epoch 3/10
75/75 [==============================] - ETA: 0s - loss: 0.9593 - accuracy: 0.6314 
Epoch 3: val_loss did not improve from 1.42729
75/75 [==============================] - 837s 11s/step - loss: 0.9593 - accuracy: 0.6314 - val_loss: 2.0642 - val_accuracy: 0.3879
Epoch 4/10
 1/75 [..............................] - ETA: 13:05 - loss: 0.7365 - accuracy: 0.7500

KeyboardInterrupt: 

## Augmentations

In [28]:
import os
import cv2
import pandas as pd
import shutil
import numpy as np
import random
import tensorflow as tf

In [4]:
def clear_directory(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path, exist_ok=True)

def save_image(image, target_path):
    os.makedirs(os.path.dirname(target_path), exist_ok=True)
    if not os.path.exists(target_path):  # Check if the file has already been copied/augmented
        cv2.imwrite(target_path, image)


In [17]:
def count_images_in_directories(directory):
    counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            counts[class_name] = len(os.listdir(class_path))
    return counts

def identify_classes_for_augmentation(counts):
    average_count = np.mean(list(counts.values()))
    return [class_name for class_name, count in counts.items() if count < average_count]

In [20]:
def horizontal_flip(image_path, label):
    image = cv2.imread(image_path)
    augmented = False  # Flag to track if the image has been augmented
    new_label = label
    # Flip the image horizontally
    image = cv2.flip(image, 1)
    augmented = True
    # Adjust the label for the augmented image
    new_label = 180 - label
    return image, augmented, new_label

def tf_adjust_brightness(image):
    return tf.image.random_brightness(image, max_delta=0.2)

def tf_adjust_contrast(image):
    return tf.image.random_contrast(image, lower=0.5, upper=1.5)


In [33]:
def save_original_images(angle_class_dir, speed_class_dir, image_dir, data):
    # Clear directories first
    clear_directory(angle_class_dir)
    clear_directory(speed_class_dir)

        # Create directories and save original images
    for _, row in data.iterrows():
        source_path = os.path.join(image_dir, f"{int(row['image_id'])}.png")
        for class_dir, class_label in [(angle_class_dir, "angle"), (speed_class_dir, "speed")]:
            target_dir = os.path.join(class_dir, str(row[class_label]))
            os.makedirs(target_dir, exist_ok=True)
            save_image(cv2.imread(source_path), os.path.join(target_dir, f"{int(row['image_id'])}.png"))

    print("Saved original files")

In [74]:
def augment_images_for_class(row, image_dir, class_dir, class_label):
    source_path = os.path.join(image_dir, f"{int(row['image_id'])}.png")
    original_target_path = os.path.join(class_dir, str(row[class_label]), f"{int(row['image_id'])}.png")
    flipped_image, _, new_label = horizontal_flip(source_path, row[class_label])

    if class_label == "angle":
        flipped_target_path = os.path.join(class_dir, str(new_label), f"{int(row['image_id'])}_flipped.png")
    else:
        flipped_target_path = os.path.join(class_dir, str(row[class_label]), f"{int(row['image_id'])}_flipped.png")
    save_image(flipped_image, flipped_target_path)
    


    # Apply brightness and contrast adjustments based on augmentation_intensity
    # for i in range(int((augmentation_intensity-1)/2)):
    #     if class_label == 'angle':
    #         if random.random() < 0.1:
    #             image_path = flipped_target_path
    #         else:
    #             image_path = original_target_path
    #     else:
    #         if random.random() < 0.5:
    #             image_path = flipped_target_path
    #         else:
    #             image_path = original_target_path

    #     image = tf.io.read_file(image_path)
    #     image = tf.image.decode_png(image, channels=3)
    #     image = tf.cast(image, tf.float32) / 255.0  # Normalize image

    #     bright_image = tf_adjust_brightness(image)
    #     bright_image_path = image_path.replace('.png', f'_bright_{i}.png')
    #     tf.keras.preprocessing.image.save_img(bright_image_path, bright_image.numpy())

    #     contrast_image = tf_adjust_contrast(image)
    #     contrast_image_path = image_path.replace('.png', f'_contrast_{i}.png')
    #     tf.keras.preprocessing.image.save_img(contrast_image_path, contrast_image.numpy())


In [61]:
def identify_classes_for_augmentation(counts):
    average_count = np.mean(list(counts.values()))
    return [class_name for class_name, count in counts.items() if count < average_count]

In [126]:
def restructuring_data(load_original, is_augment):
    data = pd.read_csv('training_norm.csv')
    data['angle'] = data['angle'] * 80 + 50
    data.loc[data['speed'] > 1, 'speed'] = 0
    
    image_dir = 'training_data/training_data/'
    angle_class_dir = 'angle_class_data_2'
    speed_class_dir = 'speed_class_data_2'

    if load_original:
        save_original_images(angle_class_dir, speed_class_dir, image_dir, data)

    # Analyze class distribution for angle and speed
    angle_counts = count_images_in_directories(angle_class_dir)
    speed_counts = count_images_in_directories(speed_class_dir)
    max_count_angle = max(angle_counts.values())
    max_count_speed = max(speed_counts.values())
    print(angle_counts, speed_counts)

    angles_for_upscaling = identify_classes_for_augmentation(angle_counts)
    speed_for_upscaling = identify_classes_for_augmentation(speed_counts)
    print(angles_for_upscaling, speed_for_upscaling)

    if is_augment:
    # Apply augmentations based on class imbalance
        for _, row in data.iterrows():
            # angle_aug_intensity = max(1, int(max_count_angle / angle_counts.get(str(row['angle']), max_count_angle)))
            # speed_aug_intensity = max(1, int(max_count_speed / speed_counts.get(str(row['speed']), max_count_speed)))
            if str(180-row['angle']) in angles_for_upscaling:
                print(row["angle"])
                # Augment images for angle class
                augment_images_for_class(row, image_dir, angle_class_dir, "angle")

            if str(row['speed']) in speed_for_upscaling:
                # Augment images for speed class
                augment_images_for_class(row, image_dir, speed_class_dir, "speed")


    print("Data restructuring complete.")

In [127]:
load_original = False
is_augment = True
restructuring_data(load_original, is_augment)

{'65.0': 1245, '50.0': 95, '75.0': 2220, '115.0': 1147, '130.0': 95, '85.0': 1467, '105.0': 2007, '120.0': 400, '95.0': 1609, '80.0': 2391, '110.0': 2123, '125.0': 93, '90.0': 2046, '100.0': 1963, '60.0': 400, '70.0': 2227, '55.0': 93} {'1.0': 10402, '0.0': 6782}
['65.0', '50.0', '115.0', '130.0', '120.0', '125.0', '60.0', '55.0'] ['0.0']
115.0
60.0
120.0
115.0
120.0
115.0
115.0
115.0
115.0
115.0
120.0
60.0
115.0
120.0
125.0
125.0
120.0
60.0
120.0
120.0
115.0
50.0
60.0
115.0
115.0
120.0
115.0
65.0
115.0
120.0
115.0
120.0
115.0
115.0
115.0
125.0
120.0
125.0
125.0
115.0
65.0
115.0
55.0
115.0
115.0
115.0
120.0
115.0
125.0
115.0
115.0
55.0
115.0
115.0
115.0
60.0
115.0
115.0
115.0
115.0
115.0
125.0
120.0
130.0
60.0
115.0
115.0
115.0
115.0
115.0
60.0
115.0
120.0
115.0
125.0
115.0
60.0
60.0
120.0
115.0
115.0
120.0
115.0
120.0
115.0
115.0
115.0
115.0
120.0
120.0
60.0
115.0
115.0
115.0
115.0
115.0
120.0
115.0
115.0
115.0
115.0
115.0
115.0
115.0
115.0
115.0
50.0
50.0
120.0
55.0
115.0
115.0
120.0

KeyboardInterrupt: 

In [129]:
angle_class_dir = 'angle_class_data_1'
speed_class_dir = 'speed_class_data_1'
angle_counts = count_images_in_directories(angle_class_dir)
speed_counts = count_images_in_directories(speed_class_dir)

print(angle_counts, speed_counts)

{'65.0': 98, '50.0': 60, '75.0': 213, '115.0': 1147, '130.0': 35, '85.0': 1467, '105.0': 2007, '120.0': 301, '95.0': 1609, '80.0': 428, '110.0': 2123, '125.0': 65, '90.0': 2046, '100.0': 1963, '60.0': 99, '70.0': 104, '55.0': 28} {'1.0': 10402, '0.0': 3391}


In [60]:
angle_counts = count_images_in_directories(angle_class_dir)
speed_counts = count_images_in_directories(speed_class_dir)

print(angle_counts, speed_counts)

{'65.0': 512, '50.0': 469, '75.0': 727, '115.0': 1291, '130.0': 399, '85.0': 1467, '105.0': 2274, '120.0': 816, '95.0': 1609, '80.0': 814, '110.0': 2263, '125.0': 485, '90.0': 2046, '100.0': 2437, '60.0': 792, '70.0': 532, '55.0': 371} {'1.0': 10402, '0.0': 5202}


In [ ]:
angle_counts = count_images_in_directories(angle_class_dir)
speed_counts = count_images_in_directories(speed_class_dir)

print(angle_counts, speed_counts)

In [90]:
from tensorflow.keras.models import load_model

In [ ]:
def load_image(
    path,
    image_size,
    num_channels,
    interpolation,
    data_format,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
):
    """Load an image from a path and resize it."""
    img = tf.io.read_file(path)
    img = tf.image.decode_image(
        img, channels=num_channels, expand_animations=False
    )

    if pad_to_aspect_ratio and crop_to_aspect_ratio:
        raise ValueError(
            "Only one of pad_to_aspect_ratio, crop_to_aspect_ratio"
            " can be set to True"
        )

    if crop_to_aspect_ratio:
        from keras.backend import tensorflow as tf_backend

        if data_format == "channels_first":
            img = tf.transpose(img, (2, 0, 1))
        img = image_utils.smart_resize(
            img,
            image_size,
            interpolation=interpolation,
            data_format=data_format,
            backend_module=tf_backend,
        )
    elif pad_to_aspect_ratio:
        img = tf.image.resize_with_pad(
            img, image_size[0], image_size[1], method=interpolation
        )
        if data_format == "channels_first":
            img = tf.transpose(img, (2, 0, 1))
    else:
        img = tf.image.resize(img, image_size, method=interpolation)
        if data_format == "channels_first":
            img = tf.transpose(img, (2, 0, 1))

    if data_format == "channels_last":
        img.set_shape((image_size[0], image_size[1], num_channels))
    else:
        img.set_shape((num_channels, image_size[0], image_size[1]))
    return img

In [124]:
import tensorflow as tf
import numpy as np

def preprocess_test_image(image_path):
    image = tf.keras.utils.load_img(image_path, target_size=(192,256), color_mode= keep_aspect_ratio=True)  # Resize the image
    im = tf.image.convert_image_dtype(image, tf.float32)
    im = tf.image.resize(im, [192, 192])
    im = tf.expand_dims(im, axis=0)
    return im


In [83]:
import os

test_directory = 'test_data/test_data'
test_image_paths = [os.path.join(test_directory, fname) for fname in os.listdir(test_directory)]

In [86]:
angles = [100.0,105.0,110.0,115.0,120.0,125.0,130.0,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0]

In [94]:
angle_path = 'models/angle/resnet_best_192_20240328'
speed_path = 'models/speed/resnet_speed_best_20240328'

angle_model = load_model(angle_path)
speed_model = load_model(speed_path)

In [101]:
def preprocess_angle(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(
        img, channels=3, expand_animations=False
    )
    im = tf.image.resize(img, [192, 192])
    # im = tf.image.grayscale_to_rgb(im)
    im = tf.cast(im, tf.float32)
    im = tf.expand_dims(im, axis=0)
    return im

In [95]:
def predict_speed(image_path, model):
    image = preprocess_test_image(image_path)
    speed = model.predict(image)[0]
    # Assuming binary classification for speed, adjust as necessary
    speed_pred = np.round(speed[0]).astype(int)
    return speed_pred

In [118]:
def predict_angle(image_path, model):
    angles = [100.0,105.0,110.0,115.0,120.0,125.0,130.0,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0]
    image = preprocess_test_image(image_path)

    angle = model.predict(image)
    
    pred_angle = angles[np.argmax(angle)]
    print(pred_angle)
    return (pred_angle-50)/80

In [125]:
model = tf.keras.models.load_model('models/angle/resnet_best_192_20240328')

test_dir = os.path.abspath('test_data/test_data')
file_list = os.listdir(test_dir)
predictions = []

for file_name in sorted(file_list, key=lambda x: int(x.split('.')[0])):
    img_path = os.path.join(test_dir, file_name)
    image_id = int(file_name.split('.')[0])
    
    angle_pred = predict_angle(img_path, angle_model)
    speed_pred = predict_speed(img_path, speed_model)
    
    print(f"Image ID: {image_id}, Angle: {angle_pred}, Speed: {speed_pred}")
    predictions.append([image_id, angle_pred, speed_pred])

1/1 [==============================] - 0s 106ms/step
120.0
1/1 [==============================] - 0s 172ms/step
Image ID: 1, Angle: 0.875, Speed: 1
1/1 [==============================] - 0s 44ms/step
120.0
1/1 [==============================] - 0s 46ms/step
Image ID: 2, Angle: 0.875, Speed: 1
1/1 [==============================] - 0s 44ms/step
120.0
1/1 [==============================] - 0s 45ms/step
Image ID: 3, Angle: 0.875, Speed: 1
1/1 [==============================] - 0s 49ms/step
120.0
1/1 [==============================] - 0s 43ms/step
Image ID: 4, Angle: 0.875, Speed: 1
1/1 [==============================] - 0s 48ms/step
120.0
1/1 [==============================] - 0s 52ms/step
Image ID: 5, Angle: 0.875, Speed: 1
1/1 [==============================] - 0s 54ms/step
120.0
1/1 [==============================] - 0s 61ms/step
Image ID: 6, Angle: 0.875, Speed: 1
1/1 [==============================] - 0s 46ms/step
120.0
1/1 [==============================] - 0s 42ms/step
Image ID: 7,

In [93]:
df_pred = pd.DataFrame(predictions, columns=['image_id', 'speed', 'angle'])
df_pred.to_csv(f'submission_check.csv', index=False)

[['test_data/test_data/348.png', 100.0],
 ['test_data/test_data/412.png', 120.0],
 ['test_data/test_data/374.png', 95.0],
 ['test_data/test_data/360.png', 120.0],
 ['test_data/test_data/406.png', 100.0],
 ['test_data/test_data/638.png', 95.0],
 ['test_data/test_data/176.png', 95.0],
 ['test_data/test_data/88.png', 95.0],
 ['test_data/test_data/610.png', 100.0],
 ['test_data/test_data/604.png', 100.0],
 ['test_data/test_data/162.png', 95.0],
 ['test_data/test_data/189.png', 100.0],
 ['test_data/test_data/837.png', 95.0],
 ['test_data/test_data/77.png', 115.0],
 ['test_data/test_data/823.png', 95.0],
 ['test_data/test_data/63.png', 110.0],
 ['test_data/test_data/980.png', 120.0],
 ['test_data/test_data/758.png', 95.0],
 ['test_data/test_data/994.png', 100.0],
 ['test_data/test_data/764.png', 95.0],
 ['test_data/test_data/770.png', 95.0],
 ['test_data/test_data/943.png', 100.0],
 ['test_data/test_data/957.png', 100.0],
 ['test_data/test_data/228.png', 110.0],
 ['test_data/test_data/566.pn